In [1]:
from Feature_Extraction import *
from threading import Thread
import concurrent.futures as conc
import cv2
import os
import math
import dlib
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_real_frame_folders = [
    '../../../archive/original_sequences/youtube/c23/videos/',
]

train_fake_frame_folders = [
    '../../../archive/manipulated_sequences/Deepfakes/c23/videos/',
    '../../../archive/manipulated_sequences/Face2Face/c23/videos/',
    '../../../archive/manipulated_sequences/FaceSwap/c23/videos/',
    '../../../archive/manipulated_sequences/NeuralTextures/c23/videos/',
]

In [3]:
real_videos_paths = []
fake_videos_paths = []
real_azi_vectors = []
fake_azi_vectors = []
reduced_dim_real_vectors = []
reduced_dim_fake_vectors = []

In [4]:
# load the real videos paths into paths list
for train_frame_folder in train_real_frame_folders:
    list_of_train_data = [os.path.join(train_frame_folder, f) for f in os.listdir(
        train_frame_folder) if f.endswith('.mp4')]
    real_videos_paths += list_of_train_data

# load the fake videos paths into paths list
for train_frame_folder in train_fake_frame_folders:
    list_of_train_data = [os.path.join(train_frame_folder, f) for f in os.listdir(
        train_frame_folder) if f.endswith('.mp4')]
    fake_videos_paths.append(list_of_train_data )

### Real videos features extraction

In [5]:
# make a worker pool with max threadings 10
pool = conc.ThreadPoolExecutor(max_workers=10)

for i, video_path in enumerate(real_videos_paths):
    # Video capturing constructor
    cap = cv2.VideoCapture(video_path)

    # starting the thread
    pool.submit(video_feature_extraction, cap, real_azi_vectors, i)

    if (i + 1) % 10 == 0:
        pool.shutdown(wait=True)

Video 0
Video 1
Video 2
Video 3
Video 3 ---- Done
Video 0 ---- Done
Video 1 ---- Done
Video 2 ---- Done


In [13]:
# Extract the features into csv file
real_videos_features_df = pd.DataFrame(data = real_azi_vectors)
real_videos_features_df['Label'] = 0
real_videos_features_df.to_csv('real_videos_features_vector.csv')

### Fake videos features extraction

In [ ]:
# make a worker pool with max threadings 10
pool = conc.ThreadPoolExecutor(max_workers=10)

for i, dir in enumerate(fake_videos_paths):
    # empty the list before extraction
    fake_azi_vectors = []
    for j, video_path in enumerate(dir):
        # Video capturing constructor
        cap = cv2.VideoCapture(video_path)

        # starting the thread
        pool.submit(video_feature_extraction, cap, fake_azi_vectors, j)

        if (i + 1) % 10 == 0:
            pool.shutdown(wait=True)
    
    # extract the csv file for the current fake dir
    fake_videos_features_df = pd.DataFrame(data = fake_azi_vectors)
    fake_videos_features_df['Label'] = 0
    fake_videos_features_df.to_csv('fake_videos_features_vector_{}.csv'.format(i))